In [1]:
                                                                                                                                                                                                                                                                                                           from flask import Flask, request, redirect, url_for,send_from_directory, render_template_string
from werkzeug.utils import secure_filename
import caffe
import json
import numpy as np
import os
from sklearn import svm,metrics
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import struct
import pandas as pd
import csv
import collections

In [2]:
deployPrototxt = "/opt/swm-ml-docker/swm-ml-master/bvlc_reference_caffenet/deploy.prototxt"
caffeModel = "/opt/swm-ml-docker/swm-ml-master/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel"
featureVectorDirectory = "/opt/swm-ml-docker/data/train"
metadataDirectory = '/opt/swm-ml-docker/metadata_bak/'
imageMeanFile = '/opt/swm-ml-docker/ilsvrc_2012_mean.npy'
layerName = 'fc7'

In [3]:
print("Loading model ...")
cnn = caffe.Net(deployPrototxt, caffe.TEST, weights=caffeModel)
print("Done loading")

Loading model ...
Done loading


In [4]:
# reshape to accept one image at a time.
(_, c, w, h) = cnn.blobs['data'].data.shape
cnn.blobs['data'].reshape(1, c, w, h)
transformer = caffe.io.Transformer({'data': cnn.blobs['data'].data.shape})
transformer.set_mean('data', np.load(imageMeanFile).mean(1).mean(1))
transformer.set_transpose('data', (2,0,1))
transformer.set_raw_scale('data', 255.0)

In [5]:
with open('Consolidated.csv','r') as csvfile:
    readCSV=csv.reader(csvfile,delimiter=',')
    Primary=[]
    Secondary=[]
    Tertiary=[]
    Lining=[]
    Names=[]
    for row in readCSV:
        #print row[0],row[1],row[2],row[3],row[7]
        Primary.append(row[1])
        Secondary.append(row[2])
        Tertiary.append(row[3])
        Lining.append(row[7])
        Names.append(row[8])
        

In [29]:
counter=collections.Counter(Lining)
print counter
del Primary[0]
del Secondary[0]
del Tertiary[0]
del Lining[0]
del Names[0]
for i in Names:
    i.encode('ascii','ignore')

Counter({'Plastic': 1531, 'None': 886, 'Newspaper': 279, 'Brown Paper': 15, 'Cloth Bag': 9})


In [53]:
classIdByLabel = {"Dry" : 0, "Wet" : 1, "Reject" : 2, "Garden" : 3,"NA":4}
classIdByLabelLining={'Plastic': 0, 'None': 1, 'Newspaper': 2, 'Brown Paper': 3, 'Cloth Bag': 4}

def loadMetaData(metadataDir):
    metadataByImageHash = {}
    allFiles = os.listdir(metadataDir)
    for fileName in allFiles:
        with open("%s/%s" % (metadataDir, fileName), "r") as jsonFile:
            metadata = json.load(jsonFile)
            metadataByImageHash[metadata['Image Hash']] = metadata
    return metadataByImageHash

def loadFeatureVectors(dirPath,metadataByImageHash, d):
    allFiles = os.listdir(dirPath)
    data = np.zeros([len(allFiles), d])
    textdata = np.zeros([len(allFiles), 4])
    for i, fileName in enumerate(allFiles):
        #print (i,fileName)
        with open("%s/%s" % (dirPath, fileName), "rb") as inputFile:
            data[i] = struct.unpack('f'*d, inputFile.read())
            metadata = metadataByImageHash[fileName]
            l=[]
            val1=classIdByLabel[metadata['Primary']]
            val2=classIdByLabel[metadata['Secondary']]
            val3=classIdByLabel[metadata['Tertiary']]
            val4=classIdByLabelLining[metadata['Lining']]
            l.extend([val1,val2,val3,val4])
            textdata[i]=l
    return (data,textdata,allFiles)

def makeBinaryLabels(metadataByImageHash, allImageHashes):
    labels = []
    for imageHash in allImageHashes:
        metadata = metadataByImageHash[imageHash]
        labels.append(1 if (metadata['Secondary'] == "NA") else 0)
    return labels

In [44]:
metadataByImageHash = loadMetaData(metadataDirectory)
data,textdata,allImageHashes=loadFeatureVectors(featureVectorDirectory,metadataByImageHash,d = 4096)

In [58]:
embeddedData=np.concatenate((data,textdata),axis=1)

In [54]:
print("Training SVM")
labels = makeBinaryLabels(metadataByImageHash, allImageHashes)
clf = svm.SVC(probability=True)
clf.fit(embeddedData, labels)
print("Finished training ready to classify images"," Data size: ",len(data))

Training SVM
('Finished training ready to classify images', ' Data size: ', 2182)


In [59]:
featureVectorTestDirectory = "/opt/swm-ml-docker/data/test"
dataTest,textdataTest,allImageHashesTest = loadFeatureVectors(featureVectorTestDirectory,metadataByImageHash, d = 4096)
embeddedDataTest=np.concatenate((dataTest,textdataTest),axis=1)
labelsTest = makeBinaryLabels(metadataByImageHash, allImageHashesTest)

In [60]:
clf.score(embeddedDataTest,labelsTest)

0.91769547325102885

In [62]:
prediction=clf.predict(embeddedDataTest)

In [63]:
CM = metrics.confusion_matrix(labelsTest,prediction)

In [64]:
print CM

[[ 91  15]
 [  5 132]]
